In [1]:
#include <iostream>
#include <string>
#include <algoviz/SVG.hpp>

// Unsere eigenen Kreationen a lá Mary Shelley
#include "Vogel.hpp"
#include "Layout.hpp"
#include "Barriere.hpp"

using namespace std;

AlgoViz::clear();

In [2]:
// Spielfeld initialisieren
SVG spielfeld = SVG(800,400);

Layout layout = Layout(spielfeld);
Vogel vogel = Vogel(spielfeld);
Text screenScore;

vogel.hide();

In [3]:
// Benötigte globale Variablen:

string key = "";

int y;

int tmpRunden;
int geschwindigkeit = 0;
int score;
int counter;

vector<Barriere*> barrieren;

bool reachMid = false;
bool alreadyCrashed = false;
bool ascend = false;

// Eine Variable, um die vertikale Geschwindigkeit zu speichern:
int ascension = 5;

// Startpositionierung
int start = 50;

// Die Iterationen der Hauptschleife unten zählen:
int iteration = 0;



In [4]:
#define GROESSE_BARRIERE_OEFFNUNG 125
Barriere* erstelleBarriere() {  
    /* Die Position der Öffnung zufällig bestimmen */
    
    int bottleneckY = (rand() % (400 - GROESSE_BARRIERE_OEFFNUNG));
    return new Barriere(bottleneckY, GROESSE_BARRIERE_OEFFNUNG, 800, spielfeld);
}

In [5]:
Barriere* erstelleDynamischeBarriere() {
    return new DynamischeBarriere(GROESSE_BARRIERE_OEFFNUNG, 800, spielfeld);
}

In [6]:
void vogelAbsturz() {
    
    alreadyCrashed = true;
    key = "x";
    
    // Barrieren transparent stellen
    for (Barriere *b : barrieren) {
        b->setFill(0, 0, 0, 0);
        b->setColor(0, 0, 0, 0);
    }
            
    layout.flackern();
    layout.gameOver();
    vogel.death();
}

In [7]:
bool vogelKollidiertMitBarriere (Barriere *b) {
    // Die x-Koordinate des Vogels ist konstant 200.
    int barriere_x = b->getXCoord();
    int oeffnung_y = b->getUpperHeight();
    int oeffnung_h = b->getBottleneckHeight();

    if (barriere_x <= 200 && barriere_x > 150) {
        if (vogel.getYCoord() < oeffnung_y || vogel.getYCoord() > oeffnung_y + oeffnung_h) {
            return true;
        }
    }
    
    return false;
}

In [8]:
bool vogelKollidiert() {
    for (Barriere *b : barrieren) {
        if (vogelKollidiertMitBarriere(b)) {
            return true;
        }
    }
    
    // Überprüft die obere und untere Bildschirmgrenze:
    if (!alreadyCrashed && (vogel.getYCoord() + 20 > 400 || vogel.getYCoord() < 0)) {
        return true;
    }
    
    return false;
}

In [9]:
// Wartet 3 Sekunden und fügt dann den Vogel ins Feld ein.
AlgoViz::sleep(3000);

layout.hideWelcome();
vogel.show();

barrieren.push_back(erstelleBarriere());

In [10]:
// Unsere Hauptschleife
while (key != "x" ) {
    /* Verarbeiten von Tastatureingaben; mit der ↑-Taste wird der Vogel nach oben beschleunigt,
       mit 'x' wird das Spiel beendet. */
    key = spielfeld.lastKey();
 
    if (key == "ArrowUp") {
        // Bewegung nach oben
        y += -15;
        ascend = true; 
    }

    /* Auf Kollision überprüfen */
    if (vogelKollidiert()) {
        vogelAbsturz();
    } else {
        score++;
    }
    
    /* Vertikale Geschwindigkeit verarbeiten */
    if (ascend) {
        vogel.move(y);
        ascension -= 10;
        if (ascension <= -10) {
            ascend = false;
        }
    } else {
        // Erdanziehung: Die Geschwindigkeit wird erhöht, 9 ist der Schwellenwert
        if (ascension <= 9) {
            ascension += 2;
        }
        y += ascension;
        vogel.move(y);
    }
    
    // Rotiere den Vogel basierend auf seiner Steigung
    if (ascension < 0) { 
        vogel.rotate(-25);
    } else {  
        vogel.rotate(45);
    }
    
    // Wenn die erste Barrier den X-Wert 400 erreicht, wird die zweite Barriere erstellt (nur für den ersten Durchlauf relevant)
    if (!reachMid && barrieren[0]->getXCoord() == 400) {
        reachMid = true;
        barrieren.push_back(erstelleBarriere());
    }
    
    // Wenn die Barrieren links am Bildschirm verschwinden, werden sie zerstört und neu erzeugt
    int i = 0;
    int zu_loeschende_barriere = -1;
    for (Barriere *b : barrieren) {
        if (b->getXCoord() <= -100) {
            zu_loeschende_barriere = i;
        }
        i++;
    }
    if (zu_loeschende_barriere != -1) {
        /* alte Barriere löschen */
        Barriere *b = barrieren.at(zu_loeschende_barriere);
        barrieren.erase(barrieren.begin() + zu_loeschende_barriere);
        delete b;
        
        /* eine neu erzeugen, zufällig auswählen, ob es eine dynamische wird */
        if ((rand() % 2) == 0) {
            barrieren.push_back(erstelleDynamischeBarriere());
        } else {
            barrieren.push_back(erstelleBarriere());
        }
    }
    
    // Bewegt die Barrieren und führt ihre Update-Aktion aus
    for (Barriere *b : barrieren) {
        b->setXCoord(b->getXCoord() - 10);
        b->update();
    }
    
    screenScore = Text("Score: " + to_string(score / 7), -20, 15, &spielfeld);
    
    iteration++;
}

In [ ]:
cout << "Score: " << to_string(score / 7) << endl;